## Часто для качественного анализа аудитории используют подходы, основанные на сегментации. Используя python, нужно построить RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации нужно выбрать следующие метрики: 

R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время,

M - сумма покупок за всё время. Подробно опишу, как я создавала кластеры. 

Для каждого RFM-сегмента построю границы метрик recency, frequency и monetary для интерпретации этих кластеров. 

Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, 
frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю.

In [39]:
# объединяю таблицы

df = pd.merge(olist_customers, olist_orders, on='customer_id')
olist_rfm = pd.merge(df, olist_items, on='order_id')

In [40]:
# оставляю только нужные колонки для удобства

olist_rfm = olist_rfm[olist_rfm.order_status == 'delivered']
orders_status_rfm = olist_rfm[['customer_unique_id','order_id','order_purchase_timestamp','price']]
orders_status_rfm.head()

,customer_unique_id,order_id,order_purchase_timestamp,price
0,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,124.99
1,290c77bc529b7ac935b93aa66c333dc3,29150127e6685892b6eab3eec79f59c7,2018-01-12 20:48:24,289.00
2,060e732b5b29e8181a18229c7b0b2b5e,b2059ed67ce144a36e2aa97d2c9e9ad2,2018-05-19 16:07:45,139.94
3,259dac757896d24d7702b9acbbff3f3c,951670f92359f4fe4a63112aa7306eba,2018-03-13 16:06:38,149.94
4,345ecd01c38d18a9036ed96c73b8d066,6b7d50bd145f6fc7f33cebabd7e49d0f,2018-07-29 09:51:30,230.00


In [41]:
orders_status_rfm.dtypes

customer_unique_id                  object
order_id                            object
order_purchase_timestamp    datetime64[ns]
price                              float64
dtype: object

In [42]:
orders_status_rfm.order_purchase_timestamp = pd.to_datetime(orders_status_rfm.order_purchase_timestamp)

C:\Temp\ipykernel_13456\3242219097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_status_rfm.order_purchase_timestamp = pd.to_datetime(orders_status_rfm.order_purchase_timestamp)


In [44]:
# Разница между текущей датой и датой последнего заказа

orders_status_rfm['time_delta'] = orders_status_rfm.order_purchase_timestamp \
    - orders_status_rfm.order_purchase_timestamp.max()
orders_status_rfm.head(10)

C:\Temp\ipykernel_13456\3190704015.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_status_rfm['time_delta'] = orders_status_rfm.order_purchase_timestamp \


,customer_unique_id,order_id,order_purchase_timestamp,price,time_delta
0,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,124.99,-470 days +00:04:58
1,290c77bc529b7ac935b93aa66c333dc3,29150127e6685892b6eab3eec79f59c7,2018-01-12 20:48:24,289.00,-229 days +05:47:47
2,060e732b5b29e8181a18229c7b0b2b5e,b2059ed67ce144a36e2aa97d2c9e9ad2,2018-05-19 16:07:45,139.94,-102 days +01:07:08
3,259dac757896d24d7702b9acbbff3f3c,951670f92359f4fe4a63112aa7306eba,2018-03-13 16:06:38,149.94,-169 days +01:06:01
4,345ecd01c38d18a9036ed96c73b8d066,6b7d50bd145f6fc7f33cebabd7e49d0f,2018-07-29 09:51:30,230.00,-32 days +18:50:53
5,4c93744516667ad3b8f1fb645a3116a4,5741ea1f91b5fbab2bd2dc653a5b5099,2017-09-14 18:14:31,259.90,-349 days +03:13:54
6,addec96d2e059c80c30fe6871d30d177,36e694cf4cbc2a4803200c35e84abdc4,2018-02-19 14:38:35,14.99,-192 days +23:37:58
7,57b2a98a409812fe9618067b6b8ebe4f,1093c8304c7a003280dd34598194913d,2017-11-16 19:29:02,19.90,-286 days +04:28:25
8,1175e95fb47ddff9de6b2b06188f7e0d,1ebeea841c590e86a14a0d7a48e7d062,2018-01-18 12:35:44,25.00,-224 days +21:35:07
9,9afe194fb833f79e300e37e580171f22,7433cbcc783205509d66a5260da5b574,2018-01-08 11:22:34,99.90,-234 days +20:21:57
